# Parse Raw Data: LCIA QSAR Project
**Author:** Jacob Kvasnicka <br>
**Date:** February 24, 2023

This module should theoretically be ran once and then any feature engineering can be done separately.

In [1]:
from config_management import UnifiedConfiguration
from raw_processing.processor import RawDataProcessor

In [2]:
config_mapping_path = 'Input\configuration-mapping.json'
config = UnifiedConfiguration(config_mapping_path)

raw_processor = RawDataProcessor(config.raw_data, config.data, config.path)

## Get chemical identifiers

In [3]:
identifiers = raw_processor.get_labeled_identifiers()

identifiers

0       DTXSID5020281
1       DTXSID8020961
2       DTXSID0021834
3       DTXSID3032622
4       DTXSID2044347
            ...      
8313    DTXSID5057882
8314    DTXSID5057884
8315    DTXSID0057885
8316    DTXSID1057905
8317    DTXSID6057906
Name: dtxsid, Length: 6598, dtype: object

In [4]:
seem3_identifiers = raw_processor.get_seem3_identifiers()

seem3_identifiers

0          DTXSID9047623
1          DTXSID0052700
2         DTXSID00583560
3         DTXSID00859050
4         DTXSID00860464
               ...      
479861     DTXSID3038307
479862     DTXSID8038300
479863     DTXSID8074158
479864     DTXSID2032180
479865     DTXSID7020895
Name: DTXSID, Length: 479866, dtype: object

In [3]:
dsstox_sdf_data = raw_processor.process_from_raw('dsstox_sdf_data')

dsstox_sdf_data

[13:49:31] WARNING: not removing hydrogen atom without neighbors
[13:49:32] WARNING: not removing hydrogen atom without neighbors
[13:49:32] WARNING: not removing hydrogen atom without neighbors
[13:49:34] WARNING: not removing hydrogen atom without neighbors
[13:49:41] WARNING: not removing hydrogen atom without neighbors
[13:50:26] WARNING: not removing hydrogen atom without neighbors
[13:50:26] WARNING: not removing hydrogen atom without neighbors
[13:50:26] WARNING: not removing hydrogen atom without neighbors
[13:50:35] WARNING: not removing hydrogen atom without neighbors
[13:50:35] WARNING: not removing hydrogen atom without neighbors
[13:50:35] WARNING: not removing hydrogen atom without neighbors
[13:50:35] WARNING: not removing hydrogen atom without neighbors
[13:50:36] WARNING: not removing hydrogen atom without neighbors
[13:50:36] WARNING: not removing hydrogen atom without neighbors
[13:50:36] WARNING: not removing hydrogen atom without neighbors
[13:50:38] WARNING: not r

[13:52:09] WARNING: not removing hydrogen atom without neighbors
[13:52:27] WARNING: not removing hydrogen atom without neighbors
[13:52:29] WARNING: not removing hydrogen atom without neighbors
[13:52:29] WARNING: not removing hydrogen atom without neighbors
[13:52:31] WARNING: not removing hydrogen atom without neighbors
[13:52:31] WARNING: not removing hydrogen atom without neighbors
[13:52:33] WARNING: not removing hydrogen atom without neighbors
[13:53:04] WARNING: not removing hydrogen atom without neighbors
[13:53:05] WARNING: not removing hydrogen atom without neighbors
[13:53:08] WARNING: not removing hydrogen atom without neighbors
[13:53:08] WARNING: not removing hydrogen atom without neighbors
[13:53:09] Unusual charge on atom 0 number of radical electrons set to zero
[13:53:09] Unusual charge on atom 0 number of radical electrons set to zero
[13:53:09] WARNING: not removing hydrogen atom without neighbors
[13:53:09] WARNING: not removing hydrogen atom without neighbors
[13

[13:53:16] WARNING: not removing hydrogen atom without neighbors
[13:53:16] atom 0 has specified valence (3) smaller than the drawn valence 18.
[13:53:16] WARNING: not removing hydrogen atom without neighbors
[13:53:16] WARNING: not removing hydrogen atom without neighbors
[13:53:17] WARNING: not removing hydrogen atom without neighbors
[13:53:17] WARNING: not removing hydrogen atom without neighbors
[13:53:17] WARNING: not removing hydrogen atom without neighbors
[13:53:17] WARNING: not removing hydrogen atom without neighbors
[13:53:17] WARNING: not removing hydrogen atom without neighbors
[13:53:17] WARNING: not removing hydrogen atom without neighbors
[13:53:17] WARNING: not removing hydrogen atom without neighbors
[13:53:18] WARNING: not removing hydrogen atom without neighbors
[13:53:18] WARNING: not removing hydrogen atom without neighbors
[13:53:18] WARNING: not removing hydrogen atom without neighbors
[13:53:18] WARNING: not removing hydrogen atom without neighbors
[13:53:18] 

[13:54:19] atom 1 has specified valence (3) smaller than the drawn valence 18.
[13:54:19] atom 1 has specified valence (3) smaller than the drawn valence 18.
[13:54:24] atom 1 has specified valence (3) smaller than the drawn valence 18.
[13:54:24] atom 1 has specified valence (6) smaller than the drawn valence 36.
[13:54:24] atom 1 has specified valence (6) smaller than the drawn valence 36.
[13:54:24] atom 1 has specified valence (6) smaller than the drawn valence 36.
[13:54:25] atom 1 has specified valence (3) smaller than the drawn valence 18.
[13:54:25] atom 1 has specified valence (3) smaller than the drawn valence 18.
[13:54:25] atom 1 has specified valence (3) smaller than the drawn valence 18.
[13:54:25] WARNING: could not find number of expected rings. Switching to an approximate ring finding algorithm.
[13:54:25] atom 1 has specified valence (3) smaller than the drawn valence 18.
[13:54:25] atom 1 has specified valence (3) smaller than the drawn valence 18.
[13:54:25] atom 1 

[13:54:34] WARNING: not removing hydrogen atom without neighbors
[13:54:34] WARNING: not removing hydrogen atom without neighbors
[13:54:34] WARNING: not removing hydrogen atom without neighbors
[13:54:34] WARNING: not removing hydrogen atom without neighbors
[13:54:34] WARNING: not removing hydrogen atom without neighbors
[13:54:34] WARNING: not removing hydrogen atom without neighbors
[13:54:34] WARNING: not removing hydrogen atom without neighbors
[13:54:34] Unusual charge on atom 0 number of radical electrons set to zero
[13:54:34] Unusual charge on atom 0 number of radical electrons set to zero
[13:54:36] WARNING: not removing hydrogen atom without neighbors
[13:54:36] WARNING: not removing hydrogen atom without neighbors
[13:54:37] WARNING: not removing hydrogen atom without neighbors
[13:54:37] WARNING: not removing hydrogen atom without neighbors
[13:54:37] WARNING: not removing hydrogen atom without neighbors
[13:54:37] WARNING: not removing hydrogen atom without neighbors
[13

,DSSTox_Compound_id,DSSTox_Substance_id,CASRN,QC_Level,Preferred_name,Mol_Weight,Mol_Formula,Monoisotopic_Mass,Dashboard_URL,ID,Canonical_SMILES
0,DTXCID80649971,DTXSID10699222,10-13-9,Public_Low,"4-[2-(3,5-Dihydroxyphenyl)ethenyl]-2-methoxybe...",274.2720000000,C15H14O5,274.0841235510,https://comptox.epa.gov/dashboard/DTXSID10699222,DTXCID80649971,COc1c(O)ccc(C=Cc2cc(O)cc(O)c2)c1O
1,DTXCID00649973,DTXSID30699224,325769-78-6,Public_Low,"4-({[4-(3-Methylphenyl)-5-(pyridin-4-yl)-1,3-t...",430.4800000000,C23H18N4O3S,430.1099616300,https://comptox.epa.gov/dashboard/DTXSID30699224,DTXCID00649973,Cc1cccc(-c2nc(NC(=O)Nc3ccc(C(=O)O)cc3)sc2-c2cc...
2,DTXCID60649974,DTXSID90699225,844488-27-3,Public_Low,Ethyl 4-{1-[(2-benzylphenyl)methyl]-1H-imidazo...,431.5360000000,C26H29N3O3,431.2208918060,https://comptox.epa.gov/dashboard/DTXSID90699225,DTXCID60649974,CCOC(=O)N1CCC(C(=O)c2nccn2Cc2ccccc2Cc2ccccc2)CC1
3,DTXCID20649975,DTXSID50699226,89450-34-0,Public_Low,2-(2-Phenylethyl)-4-[2-(4-propylcyclohexyl)eth...,383.5970000000,C26H39O2,383.2955540760,https://comptox.epa.gov/dashboard/DTXSID50699226,DTXCID20649975,CCCC1CCC(CCC2CCC(C(=O)[O-])C(CCc3ccccc3)C2)CC1
4,DTXCID80649976,DTXSID10699227,365428-93-9,Public_Low,"4-[2-Ethyl-4-(3-propylphenyl)-1,3-thiazol-5-yl...",323.4600000000,C19H21N3S,323.1456188620,https://comptox.epa.gov/dashboard/DTXSID10699227,DTXCID80649976,CCCc1cccc(-c2nc(CC)sc2-c2ccnc(N)c2)c1
...,...,...,...,...,...,...,...,...,...,...,...
1114489,DTXCID70649965,DTXSID00699216,906087-50-1,Public_Low,4-Amino-N-methyl-1H-pyrazole-5-carboxamide,140.1460000000,C5H8N4O,140.0698108940,https://comptox.epa.gov/dashboard/DTXSID00699216,DTXCID70649965,CNC(=O)c1[nH]ncc1N
1114490,DTXCID90649967,DTXSID20699218,479481-55-5,Public_Low,4-Benzylpyridazin-3(2H)-one,186.2140000000,C11H10N2O,186.0793129500,https://comptox.epa.gov/dashboard/DTXSID20699218,DTXCID90649967,O=c1[nH]nccc1Cc1ccccc1
1114491,DTXCID50649968,DTXSID80699219,324519-70-2,Public_Low,4-Benzyl-3-[5-chloro-2-(propan-2-yl)pent-4-eno...,335.8300000000,C18H22ClNO3,335.1288213000,https://comptox.epa.gov/dashboard/DTXSID80699219,DTXCID50649968,CC(C)C(CC=CCl)C(=O)N1C(=O)OCC1Cc1ccccc1
1114492,DTXCID10649969,DTXSID90699220,830317-02-7,Public_Low,4-[(4-Methylanilino)methylidene]pyridin-3(4H)-one,212.2520000000,C13H12N2O,212.0949630140,https://comptox.epa.gov/dashboard/DTXSID90699220,DTXCID10649969,Cc1ccc(NC=C2C=CN=CC2=O)cc1


## Target variable: Surrogate POD [mg/(kg-d)]

In [5]:
surrogate_pods = raw_processor.process_from_raw('surrogate_pods')

surrogate_pods

,general,repro_dev
DTXSID,,
DTXSID00100074,NaN,2.524183
DTXSID00100498,1.980348,NaN
DTXSID001005033,NaN,2.124155
DTXSID001006300,1.706121,NaN
DTXSID00100756,NaN,2.411296
...,...,...
DTXSID9096313,1.240166,NaN
DTXSID90965533,NaN,1.899360
DTXSID9098147,1.968909,2.411296


## Regulatory PODs (fully adjusted to human equivalent dose) [mg(kg-d)]

See the tab “Data for Figure 5” – the CASRN are in columns A (general non-cancer) and G (repro/dev effects), and the regulatory PODs (fully adjusted to human equivalent dose) are in columns F and L.

In [6]:
%%time

reg_pods = raw_processor.process_from_raw('regulatory_pods')

reg_pods

CPU times: total: 62.5 ms
Wall time: 76.7 ms


,general,repro_dev
DTXSID,,
DTXSID5020281,-0.891,NaN
DTXSID8020961,0.286,NaN
DTXSID6026080,2.193,NaN
DTXSID0021836,-0.534,NaN
DTXSID3020596,2.028,NaN
...,...,...
DTXSID9021762,NaN,2.682
DTXSID0039229,NaN,1.438
DTXSID5021386,NaN,0.191


## Experimental LD50 values

In [8]:
%%time 

ld50s = raw_processor.process_from_raw('experimental_ld50s')

ld50s

CPU times: total: 781 ms
Wall time: 844 ms


,median_LD50
DTXSID,
DTXSID5020281,138.120754
DTXSID8020961,126.969502
DTXSID0021834,69.543651
DTXSID2044347,420.473799
DTXSID9059204,518.581177
...,...
DTXSID60469235,73.338453
DTXSID2033447,134.034040
DTXSID3041794,366.640543


## CompTox features: OPERA + TEST predictions

In [9]:
%%time

comptox_features = raw_processor.process_from_raw('comptox_features')

comptox_features

CPU times: total: 109 ms
Wall time: 168 ms


,AVERAGE_MASS,48HR_DAPHNIA_LC50_MOL/L_TEST_PRED,DENSITY_G/CM^3_TEST_PRED,DEVTOX_TEST_PRED,96HR_FATHEAD_MINNOW_MOL/L_TEST_PRED,FLASH_POINT_DEGC_TEST_PRED,AMES_MUTAGENICITY_TEST_PRED,ORAL_RAT_LD50_MOL/KG_TEST_PRED,SURFACE_TENSION_DYN/CM_TEST_PRED,THERMAL_CONDUCTIVITY_MW/(M*K)_TEST_PRED,...,BIODEGRADATION_HALF_LIFE_DAYS_DAYS_OPERA_PRED,BOILING_POINT_DEGC_OPERA_PRED,HENRYS_LAW_ATM-M3/MOLE_OPERA_PRED,OPERA_KM_DAYS_OPERA_PRED,OCTANOL_AIR_PARTITION_COEFF_KOA_OPERA_PRED,SOIL_ADSORPTION_COEFFICIENT_KOC_L/KG_OPERA_PRED,OCTANOL_WATER_PARTITION_P_OPERA_PRED,MELTING_POINT_DEGC_OPERA_PRED,VAPOR_PRESSURE_MMHG_OPERA_PRED,WATER_SOLUBILITY_MOL/L_OPERA_PRED
DTXSID,,,,,,,,,,,,,,,,,,,,,
DTXSID5020281,157.550,1.081430e-04,1.372,0.323,5.714790e-05,109.227,0.966,0.007079,NaN,137.950,...,4.84081,242.050,4.947790e-06,0.332365,2.124320e+04,507.6930,246.342872,82.9942,2.203470e-02,1.317610e-03
DTXSID8020961,138.126,6.486340e-05,1.309,0.250,4.385310e-04,141.229,0.642,0.009141,NaN,162.043,...,6.64833,331.672,1.265030e-09,0.306715,3.734737e+06,75.9962,24.436556,145.5400,3.265100e-06,4.815930e-03
DTXSID0021834,139.110,5.395110e-05,1.376,0.281,1.472310e-04,117.588,0.490,0.005140,NaN,152.108,...,4.09570,278.910,4.207650e-10,0.251436,1.297807e+06,233.1200,81.678923,113.5640,9.970590e-05,9.679290e-02
DTXSID3032622,99.089,NaN,1.276,0.639,NaN,36.844,0.372,0.004457,NaN,155.108,...,4.27009,215.029,9.095360e-08,0.288388,5.393492e+04,10.0605,2.852542,86.5110,1.396390e-03,8.066960e-01
DTXSID2044347,150.177,1.312200e-04,1.072,0.621,2.904020e-04,98.358,0.305,0.004887,36.672,139.705,...,4.91679,248.981,9.683370e-08,0.279812,1.183477e+05,98.5113,55.324819,37.4772,6.500820e-03,1.201570e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DTXSID6057879,345.460,1.291220e-05,1.232,0.860,9.183330e-07,271.676,0.563,0.001766,NaN,NaN,...,6.00889,417.425,1.882080e-07,1.137960,3.325830e+10,881.4960,5365.126916,181.8440,1.564390e-08,4.541010e-07
DTXSID5057880,353.860,7.144960e-06,1.256,0.846,1.023290e-06,261.844,0.204,NaN,NaN,NaN,...,3.35362,415.812,2.941970e-07,1.655430,4.403520e+10,961.6230,892.585855,125.4470,6.420140e-08,5.643740e-07
DTXSID5057882,300.150,2.065380e-07,1.363,0.679,7.211070e-07,187.442,0.313,NaN,NaN,138.484,...,3.54031,337.893,2.333810e-07,3.627750,4.228633e+10,8156.8700,7006.481829,161.2890,2.468280e-07,1.982760e-06


## OPERA 2.9 features

In [10]:
AD_flags, opera_features = raw_processor.process_from_raw('opera_features')

opera_features

,CERAPP_Ago_pred_discrete,CERAPP_Anta_pred_discrete,CERAPP_Bind_pred_discrete,CoMPARA_Ago_pred_discrete,CoMPARA_Anta_pred_discrete,CoMPARA_Bind_pred_discrete,CATMoS_LD50_pred,FUB_pred,Clint_pred,CACO2_pred,...,Sp3Sp2HybRatio,nbRotBd_discrete,nbHBdAcc_discrete,ndHBdDon_discrete,nbLipinskiFailures_discrete,TopoPolSurfAir,MolarRefract,CombDipolPolariz,VP_pred,WS_pred
DTXSID,,,,,,,,,,,,,,,,,,,,,
DTXSID5020281,0.0,0.0,0.0,0.0,0.0,0.0,236.0,0.20,399.37,NaN,...,0.000000,1.0,0.0,0.0,0.0,43.14,9.5056,0.900,2.187762e-02,1.318257e-03
DTXSID8020961,0.0,0.0,0.0,0.0,0.0,0.0,787.0,0.33,23.59,NaN,...,0.000000,1.0,1.0,1.0,0.0,69.16,8.1121,1.201,3.311311e-06,4.786301e-03
DTXSID0021834,0.0,0.0,0.0,0.0,0.0,0.0,228.0,0.15,25.94,NaN,...,0.000000,1.0,0.0,1.0,0.0,63.37,6.4546,1.065,1.000000e-04,9.772372e-02
DTXSID3032622,0.0,0.0,0.0,0.0,0.0,0.0,2446.0,0.93,4.35,NaN,...,0.250000,0.0,1.0,1.0,0.0,38.33,24.3643,0.909,1.380384e-03,8.128305e-01
DTXSID2044347,0.0,0.0,0.0,0.0,0.0,0.0,2606.0,0.15,0.00,-4.92,...,0.222222,2.0,1.0,0.0,0.0,26.30,18.0862,1.185,6.606934e-03,1.412538e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DTXSID10855048,0.0,0.0,0.0,0.0,1.0,1.0,2702.0,NaN,NaN,NaN,...,0.000000,2.0,0.0,0.0,1.0,9.23,44.6146,1.900,6.760830e-08,1.445440e-09
DTXSID5047902,0.0,0.0,0.0,0.0,0.0,0.0,1587.0,NaN,0.67,NaN,...,1.000000,1.0,2.0,2.0,0.0,46.25,16.2111,0.530,5.011872e-02,1.621810e+01
DTXSID30785570,0.0,0.0,0.0,0.0,1.0,1.0,2702.0,NaN,NaN,NaN,...,0.000000,2.0,0.0,0.0,1.0,9.23,44.6146,1.948,8.317638e-08,7.943282e-10
